<a href="https://colab.research.google.com/github/vrmusketeers/FYP-2021/blob/master/notebooks/neural_net/Inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
!unzip '/content/drive/Shareddrives/Final Project/FmriTrainTestData/Inception-Data/Output.zip' -d '/content/drive/Shareddrives/Final Project/FmriTrainTestData/Inception-Data/'

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/Shareddrives/Final Project/FmriTrainTestData/Inception-Data/Output/train/2/203_9.png  
  inflating: /content/drive/Shareddrives/Final Project/FmriTrainTestData/Inception-Data/__MACOSX/Output/train/2/._203_9.png  
  inflating: /content/drive/Shareddrives/Final Project/FmriTrainTestData/Inception-Data/Output/train/2/255_3.png  
  inflating: /content/drive/Shareddrives/Final Project/FmriTrainTestData/Inception-Data/__MACOSX/Output/train/2/._255_3.png  
  inflating: /content/drive/Shareddrives/Final Project/FmriTrainTestData/Inception-Data/Output/train/2/598_6.png  
  inflating: /content/drive/Shareddrives/Final Project/FmriTrainTestData/Inception-Data/__MACOSX/Output/train/2/._598_6.png  
  inflating: /content/drive/Shareddrives/Final Project/FmriTrainTestData/Inception-Data/Output/train/2/664_6.png  
  inflating: /content/drive/Shareddrives/Final Project/FmriTrainTestData/Inception-Data/__MACOSX/Output/train/2

In [18]:
!rm -rf '/content/drive/Shareddrives/Final Project/FmriTrainTestData/Inception-Data/Output/__MACOSX'
!rm -rf '/content/drive/Shareddrives/Final Project/FmriTrainTestData/Inception-Data/Output/train/.ipynb_checkpoints'
!rm -rf '/content/drive/Shareddrives/Final Project/FmriTrainTestData/Inception-Data/Output/test/.ipynb_checkpoints'
!rm -rf '/content/drive/Shareddrives/Final Project/FmriTrainTestData/Inception-Data/Output/val/.ipynb_checkpoints'

In [19]:
import os 
import zipfile 
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
import matplotlib.pyplot as plt

In [20]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2,shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [22]:
train_generator = train_datagen.flow_from_directory('/content/drive/Shareddrives/Final Project/FmriTrainTestData/Inception-Data/Output/train', batch_size = 20, class_mode = 'binary', target_size = (150, 150))
validation_generator = test_datagen.flow_from_directory('/content/drive/Shareddrives/Final Project/FmriTrainTestData/Inception-Data/Output/test', batch_size = 20, class_mode = 'binary', target_size = (150, 150))

Found 6968 images belonging to 2 classes.
Found 871 images belonging to 2 classes.


In [23]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(input_shape = (150, 150, 3), include_top = False, weights = 'imagenet')

In [24]:
for layer in base_model.layers:
    layer.trainable = False

In [28]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.15)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])

In [30]:
inc_history = model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 100)

Epoch 1/100
100/100 [==============================] - 14s 138ms/step - loss: 0.6912 - acc: 0.5478 - val_loss: 0.7154 - val_acc: 0.5327
Epoch 2/100
100/100 [==============================] - 14s 137ms/step - loss: 0.7006 - acc: 0.5448 - val_loss: 0.6951 - val_acc: 0.5327
Epoch 3/100
100/100 [==============================] - 14s 137ms/step - loss: 0.6999 - acc: 0.5185 - val_loss: 0.7126 - val_acc: 0.4638
Epoch 4/100
100/100 [==============================] - 14s 137ms/step - loss: 0.6991 - acc: 0.5115 - val_loss: 0.7058 - val_acc: 0.4730
Epoch 5/100
100/100 [==============================] - 14s 142ms/step - loss: 0.7010 - acc: 0.5215 - val_loss: 0.6948 - val_acc: 0.5201
Epoch 6/100
100/100 [==============================] - 14s 139ms/step - loss: 0.6982 - acc: 0.5230 - val_loss: 0.6913 - val_acc: 0.5419
Epoch 7/100
100/100 [==============================] - 14s 137ms/step - loss: 0.6957 - acc: 0.5312 - val_loss: 0.6905 - val_acc: 0.5465
Epoch 8/100
100/100 [===========================